# Read Libs

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests as req
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time



# display and output settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

display(HTML('''
<style>
    .container { width:85% !important; }
</style>
'''))

pd.set_option('display.min_rows', 100)

In [18]:
# Specify the path to the ChromeDriver executable
chrome_driver_path = '../drivers/chromedriver_arm'
# firefox_driver_path = '../drivers/geckodriver'

#link for solution to cannot open chromedriver
# https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
# xattr -d com.apple.quarantine

# Initialize the Chrome WebDriver
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Required for headless mode to work properly on Windows
# options.add_argument('--no-sandbox')  # Bypass OS security model, required for running as root user
# options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

# driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

driver = webdriver.Chrome(executable_path=chrome_driver_path)


# Open the Weather Underground history page
driver.get("https://www.wunderground.com/history")

# Wait for the page to load and locate the city input field
city_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "historySearch"))
)

# Enter the city name
city_input.clear()
city_input.send_keys("Augusta, GA")
time.sleep(2)
city_input.send_keys(Keys.RETURN)



/var/folders/sj/kb7kr7w572z54pgv0pftb9v00000gn/T/ipykernel_55807/1777907275.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


In [19]:
# month_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[1]"
# month_selection = driver.find_element(By.XPATH, month_selection_xpath)

# day_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[2]"
# day_selection = driver.find_element(By.XPATH, day_selection_xpath)

# year_selection_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/select[3]"
# year_selection = driver.find_element(By.XPATH, year_selection_xpath)

# month_selection.click()
# month_selection.send_keys("January")

# day_selection.click()
# day_selection.send_keys("1")

# year_selection.click()
# year_selection.send_keys("2020")
# year_selection.send_keys(Keys.RETURN)

from selenium.common.exceptions import TimeoutException
driver.set_page_load_timeout(10)

# Submit the form
submit_button_xpath = "/html/body/app-root/app-history-search/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div/div/div/div/form/lib-date-selector/div/input"
submit_button = driver.find_element(By.XPATH, submit_button_xpath)


submit_button.click()

title_xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[1]/lib-city-header/div[1]/div/h1/span[1]"

try:
    title_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, title_xpath))
    )
except TimeoutException:
    driver.execute_script("window.stop();")
    print("Timed out waiting for page to load")
    current_url = driver.current_url
    print(current_url)


    



Timed out waiting for page to load
https://www.wunderground.com/history/daily/KAGS/date/2024-10-13


In [15]:
current_url

'https://www.wunderground.com/subscribe?tpcc=mktgWU-history-subs-nav_goAdFree'

In [34]:
# Find the table element on the webpage
daily_table_xpath = "/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div[2]/section/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table"
table = driver.find_element(By.XPATH, daily_table_xpath)

# Extract the HTML content of the table
table_html = table.get_attribute('outerHTML')

# Use pandas to read the HTML content into a DataFrame
df_hourly_weather = pd.read_html(table_html)[0]
